<a href="https://colab.research.google.com/github/pokem1402/section_project_one/blob/main/EDA_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Loading Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

os.chdir("drive/My Drive/dataset")

In [ ]:
!ls

vgames2.csv


In [ ]:
import pandas as pd

df_original = pd.read_csv("vgames2.csv")

# 2. EDA

In [ ]:
df_original.head()

,Unnamed: 0,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,1,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
1,2,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
2,3,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
3,4,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
4,5,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04


## 1) 데이터에 있는 이상치 발견 및 수정

In [ ]:
"""
DataFrame.iterrows()
 : row 값에 대해 loop를 수행하게 해주는 함수.
"""

'\nDataFrame.iterrows()\n : row 값에 대해 loop를 수행하게 해주는 함수.\n'

In [ ]:
# (1) 불필요한 index 제거
df = df_original.drop(axis=1, columns="Unnamed: 0")

In [ ]:
# (2) 중복 제거
df.drop_duplicates(inplace = True)

In [ ]:
# (3) Year를 Int 형식으로 바꿀 수 없는 이유를 찾고, 수정
import numpy as np
print("The number of na : ", df.Year.isna().sum())
# df.Year.dropna().astype("int") # nan 만 존재.
nan_list = [[idx, df.loc[idx]["Name"], df.loc[idx]["Platform"]] for idx, _ in enumerate(df.Year.isna()) if _] # nan game list

The number of na :  270


In [ ]:
# (4) 데이터 shape 확인
df.shape

(16597, 9)

### (2) nan 값이 들어있는 데이터의 처리 방식

1.   삭제 (dropna)
2.   관련 데이터를 찾아서 해당 내용을 수정하기 (modify)
3.   무시 (ignore)

In [ ]:
# (2) 1. 삭제를 할 경우 데이터가 얼마나 줄어들까.
print(df.isna().any(axis=1).sum())

356


In [ ]:
# (2) 2. 어느 attribute에 nan 값이 존재하는가
print(df.isna().any(axis=0))

Name           False
Platform       False
Year            True
Genre           True
Publisher       True
NA_Sales       False
EU_Sales       False
JP_Sales       False
Other_Sales    False
dtype: bool


#### Web에서 게임 이름을 입력하면 Year와 Publisher 그리고 Genre를 얻기

Reference : https://www.vgchartz.com/

In [ ]:
# nan이 존재하는 데이터의 index
nan_idx = df[df.isna().any(axis=1)].index
nan_idx

Int64Index([   31,    50,   109,   272,   273,   358,   408,   430,   548,
              675,
            ...
            16130, 16195, 16240, 16275, 16278, 16425, 16431, 16432, 16555,
            16587],
           dtype='int64', length=356)

##### 이름을 입력하면 크롤링하는 함수

In [ ]:
import re
import lxml.html as lh
import requests
import traceback
import sys
import time

def debugging(message, Debug):
  if Debug:
    print(message)

def get_data_from_web(game_name, Year = True, Publisher = True, Genre = True, Debug=False, sleep_time = 10, wait_count = 5, return_value = None):
  print(game_name, end = ' ')

  if return_value is None:
    return_value = {"Name":game_name, "Complete" : False, "error": False}
  try:
    if(Year or Publisher or Genre):
      if Year:
        return_value["Year"] = -1
      if Genre:
        return_value["Genre"] = -1
      if Publisher:
        return_value["Publisher"] = -1
    

      game_name_ = game_name.replace(" ", "+")

      game_base_url = "https://www.vgchartz.com/game/"
      base_url = "https://www.vgchartz.com/games/games.php?name="
      search_attr ="&keyword=&console=&region=All&developer=&publisher=&goty_year=&genre=&boxart=Both&banner=Both&ownership=Both&showmultiplat=No&results=50&order=Sales&showtotalsales=0&showpublisher=0&showpublisher=1&showvgchartzscore=0&shownasales=0&showdeveloper=0&showcriticscore=0&showpalsales=0&showreleasedate=0&showreleasedate=1&showuserscore=0&showjapansales=0&showlastupdate=0&showothersales=0&showshipped=0"
      resp = requests.get(base_url+game_name+search_attr)
      
      count = 0
      while(not resp.ok):
        print("not ok")
        time.sleep(sleep_time)
        resp = requests.get(base_url+game_name+search_attr)
        count+=1
        if(count > wait_count):
          return_value["timeout"] = True
          return return_value

      doc = lh.fromstring(resp.content)

      def get_str(element):
        s = str(element.text_content())
        s = re.sub(r"(\s\s+)", "", s)
        return s 

      def get_year(year):
        if len(year) == 1:
          year = '0'+year
        return int(('19' if int(year) > 70 else '20') + year)

      names = doc.xpath('//tr/td[3]')
      # idx = -1;
      idxs = []
      for i,name in enumerate(names[4:]):
        
        name_ = get_str(name)
        extra_string_start = name_.find("Read the review")

        if (extra_string_start != -1):
          name_ = name_[:extra_string_start]
        if(name_ == game_name) or (name_.lower() == game_name.lower()) or (re.sub("[.!]", "", name_.lower()) == re.sub("[.!]", "", game_name.lower())):
          # idx = i
          idxs.append(i)
      #
      # year
      #
      flag_year = False
      idx = -1
      if Year:
        years = doc.xpath('//tr/td[6]')
        for idx_ in idxs:
          year = get_str(years[idx_+3])
          year = re.sub(" ", "", year)
          year_ = year[-2:]
          if(year_ != "/A"):
            print(" Year : ", end="")
            year_ = get_year(year_)
            return_value["Year"] = year_ 
            print(year_, end = ' ')
            idx = idx_
            break
        if idx == -1:
          flag_year = True

      #
      # Publisher
      #      
      if Publisher:
        print(" Publisher : " , end="")
        publishers = doc.xpath('//tr/td[5]')
        if idx != -1:
          publisher = get_str(publishers[idx+3])
          return_value["Publisher"] = publisher
        else:
          for idx_ in idxs:
            publisher = get_str(publishers[idx_+3])
            if publisher == "Unknown":
              continue
            else:
              return_value["Publisher"] = publisher
              idx = idx_
        if return_value["Publisher"] == -1:
          idx = 0
          return_value["Publisher"] = "Unknown"
        print(publisher, end = ' ')

      #
      # Genre
      #
      if Genre or flag_year:
        hrefs = doc.xpath('//tr/td[3]/a/@href')
        game_name_ = game_name.lower()
        game_name_ = re.sub("[!:\\(\\)$,./']", "", game_name_)
        game_name_ = re.sub("-", " ", game_name_)
        game_name_ = re.sub("[ ]{1,}", " ", game_name_)

        if idx != -1:
          # for href in hrefs:
          href_part = hrefs[idx].split("/") 
          # remove special character
          ss = href_part[5].replace("-", " ")
          if(ss == game_name_):
            game_info_url = hrefs[idx]
        else:
          for idx_ in idxs:

            href_part = hrefs[idx_].split("/")
            if(len(href_part) < 5):
              continue 
            # remove special character
            ss = href_part[5].replace("-", " ")
            game_name_ = game_name_.replace("&", "amp")
            if(ss == game_name_):
              game_info_url = hrefs[idx_]
        print(game_info_url, end=" ")

        resp_ = requests.get(game_info_url)

        count = 0
        while(not resp_.ok):
          print("not ok")
          time.sleep(sleep_time)
          resp = requests.get(base_url+game_name+search_attr)
          count+=1
          if(count > wait_count):
            return_value["timeout"] = True
            return return_value
        
        doc_ = lh.fromstring(resp_.content)
        txt = doc_.text_content()
        start = txt.find("Genre")+len("Genre")
        if Genre:
          print(" Genre : ", end="")
          end = txt.find("Other Versions")
          end = end if end != -1 else txt.find("Release Dates", start)
          genre = txt[start:end]
          return_value["Genre"] = genre
          print(genre, end = ' ')
        if flag_year:
          print(" Year : ", end="")
          start = txt.find("Release Dates",start)+len("Release Dates")
          end = txt.find("Community Stats")
          years_ = re.findall("\d\d\/\d\d\/\d\d", txt[start:end])
          year = get_year(str(min([int(y_[-2:]) for y_ in years_])))
          return_value["Year"] = year
          print(year, end=' ')
      
      return_value["Complete"] = True
      return return_value

  except BaseException as e:
    # Get current system exception
    ex_type, ex_value, ex_traceback = sys.exc_info()

    # Extract unformatter stack traces as tuples
    trace_back = traceback.extract_tb(ex_traceback)

    # Format stacktrace
    stack_trace = list()

    for trace in trace_back:
        stack_trace.append("File : %s , Line : %d, Func.Name : %s, Message : %s" % (trace[0], trace[1], trace[2], trace[3]))

    if(Debug):
      print("Exception type : %s " % ex_type.__name__)
      print("Exception message : %s" %ex_value)
      print("Stack trace : %s" %stack_trace)
      return_value["error"]= True
      return return_value
    else:
      return_value["error"]= True
      return_value["error_msg"] = []
      return_value["error_msg"].append("Exception type : %s " % ex_type.__name__)
      return_value["error_msg"].append("Exception message : %s" %ex_value)
      return_value["error_msg"].append("Stack trace : %s" %stack_trace)
      return return_value

In [ ]:
# Test
get_data_from_web("Tour de France 2011", Year=True, Publisher=False, Genre = True, Debug=True);

#### 저장된 리스트 혹은 빈 리스트를 넣고 인덱스를 넣으면 df로부터 크롤링을 수행하게하는 함수

In [ ]:
def find_missing_value(searched_values, missed_idx):

  n = len(searched_values)

  for i, idx in enumerate(missed_idx):
    print(i, idx, end=" ")


    Year = np.isnan(df.loc[idx].Year)
    Publisher = False if type(df.loc[idx].Publisher) == str else np.isnan(df.loc[idx].Publisher)
    Genre = False if type(df.loc[idx].Genre) == str else np.isnan(df.loc[idx].Genre)


    if (i+1 > n): # not visited : append
      value = get_data_from_web(df.loc[idx].Name,
                                Year = Year,
                                Publisher = Publisher,
                                Genre = Genre,
                                Debug=True)

      searched_values.append([idx, value])
      n += 1
      
    else: # already visited

      keys = searched_values[i][1].keys()

      # Year : nan and if year value is found already -> no need to find
      # Year : nan and if year value is not found -> we need to find
      Year = Year and ("Year" not in keys)
      Publisher = Publisher and ("Publisher" not in keys)
      Genre = Genre and ("Genre" not in keys)

      searched_values[i] = get_data_from_web(df.loc[idx].Name,
                                            Year = Year,
                                            Publisher = Publisher,
                                            Genre = Genre,
                                            Debug=True,
                                            return_value = searched_values[i][1])

    print("")
  return searched_values

#### nan을 가진 인덱스로부터 nan 값에 해당하는 값을 크롤링

- 에러가 나는 항목은 해당 사이트에서 찾을 수 없는 데이터

In [ ]:
searched_values = find_missing_value([], nan_idx)

0 31 Rock Band  Year : 2007 
1 50 Ape Escape: On the Loose https://www.vgchartz.com/game/39001/ape-escape-on-the-loose/?region=All  Genre : Platform 
2 109 Yu-Gi-Oh! 5D's Wheelie Breakers (JP sales)  Year : 2009 
3 272 UK Truck Simulator  Publisher : Excalibur Publishing 
4 273 Jewel Link Chronicles: Mountains of Madness  Year : 2012 
5 358 Majesty 2: The Fantasy Kingdom Sim  Year : 2009 
6 408 NBA Ballers https://www.vgchartz.com/game/1509/nba-ballers/?region=All  Genre : Sports 
7 430 Famista 64 https://www.vgchartz.com/game/3272/famista-64/?region=All  Year : 1997 
8 548 Transworld Surf  Year : 2002 
9 675 Battle vs. Chess Exception type : UnboundLocalError 
Exception message : local variable 'game_info_url' referenced before assignment
Stack trace : ['File : <ipython-input-3-adfb243c89e3> , Line : 140, Func.Name : get_data_from_web, Message : print(game_info_url, end=" ")']

10 808 Major League Baseball 2K6  Year : 2006 
11 818 Twisted Metal: Small Brawl  Year : 2001 
12 952 Tiger 

#### 연결 불량등 Time out의 이유로 데이터를 추출하지 못하는 경우 및 다른 이유로 데이터를 추출하지 못한 데이터를 재획득

In [ ]:
searched_values_ = searched_values.copy()

In [ ]:
for i, value in enumerate(searched_values_):

  if not value[1]["Complete"]:
    idx = value[0]
    Year = np.isnan(df.loc[idx].Year)
    Publisher = False if type(df.loc[idx].Publisher) == str else np.isnan(df.loc[idx].Publisher)
    Genre = False if type(df.loc[idx].Genre) == str else np.isnan(df.loc[idx].Genre)
    searched_values_[i] = [value[0], get_data_from_web(df.loc[idx].Name,
                                            Year = Year,
                                            Publisher = Publisher,
                                            Genre = Genre,
                                            Debug=True,
                                            return_value = searched_values[i][1])]
    print("")

Battle vs. Chess Exception type : UnboundLocalError 
Exception message : local variable 'game_info_url' referenced before assignment
Stack trace : ['File : <ipython-input-3-adfb243c89e3> , Line : 140, Func.Name : get_data_from_web, Message : print(game_info_url, end=" ")']

Pokemon X/Pokemon Y Exception type : UnboundLocalError 
Exception message : local variable 'game_info_url' referenced before assignment
Stack trace : ['File : <ipython-input-3-adfb243c89e3> , Line : 140, Func.Name : get_data_from_web, Message : print(game_info_url, end=" ")']

Final Fantasy XI Exception type : UnboundLocalError 
Exception message : local variable 'game_info_url' referenced before assignment
Stack trace : ['File : <ipython-input-3-adfb243c89e3> , Line : 140, Func.Name : get_data_from_web, Message : print(game_info_url, end=" ")']

Writing and Speaking Beautiful Japanese DS Exception type : UnboundLocalError 
Exception message : local variable 'game_info_url' referenced before assignment
Stack trace :

#### 마지막으로 불량 데이터 확인

In [ ]:
for i, value in enumerate(searched_values):

  if not value[1]["Complete"]:
    print(value)

[675, {'Name': 'Battle vs. Chess', 'Complete': False, 'error': False, 'Year': -1}]
[1971, {'Name': 'Pokemon X/Pokemon Y', 'Complete': False, 'error': False, 'Genre': -1}]
[2026, {'Name': 'Final Fantasy XI', 'Complete': False, 'error': False, 'Year': -1}]
[2408, {'Name': 'Writing and Speaking Beautiful Japanese DS', 'Complete': False, 'error': False, 'Year': -1}]
[3184, {'Name': 'Action Man-Operation Extreme', 'Complete': False, 'error': False, 'Year': -1, 'Publisher': 'Unknown'}]
[5945, {'Name': 'Medabots: Metabee', 'Complete': False, 'error': False, 'Genre': -1}]
[9298, {'Name': 'Battle vs. Chess', 'Complete': False, 'error': False, 'Year': -1}]
[9684, {'Name': 'Dragon Ball Z: Budokai Tenkaichi 2 (JP sales)', 'Complete': False, 'error': False, 'Year': -1, 'Publisher': 'Unknown'}]
[10156, {'Name': 'The Legend of Zelda: The Minish Cap(weekly JP sales)', 'Complete': False, 'error': False, 'Year': -1, 'Publisher': 'Unknown'}]
[11711, {'Name': 'Hakuouki: Shinsengumi Kitan', 'Complete': Fal

### (3) 데이터 교정

In [ ]:
df_ = df.copy()

In [ ]:
df_.Year.loc[30]

2011.0

In [ ]:
for idx, value in searched_values:

  Name = value["Name"]
  if ("Publisher" in value.keys()):
    if value["Publisher"] != -1:

      df_.at[idx,"Publisher"] = value["Publisher"]

  if ("Genre" in value.keys()):
    if value["Genre"] != -1:
      df_.at[idx,"Genre"] = value["Genre"]
  
  if ("Year" in value.keys()):
    if value["Year"] != -1:
      df_.at[idx,"Year"] = value["Year"]

df_.isna().sum()

Name            0
Platform        0
Year           12
Genre           2
Publisher       0
NA_Sales        0
EU_Sales        0
JP_Sales        0
Other_Sales     0
dtype: int64

#### 교정되지 못한 데이터 제거

In [ ]:
df_.dropna(inplace = True)
print("제거된 데이터의 수 : ", df.shape[0] - df_.shape[0])

제거된 데이터의 수 :  14


Name           0
Platform       0
Year           0
Genre          0
Publisher      0
NA_Sales       0
EU_Sales       0
JP_Sales       0
Other_Sales    0
dtype: int64

### (4) 교정된 데이터 저장

In [ ]:

drive.mount('/content/drive')
path = '/content/drive/My Drive/dataset/vgames2_modified.csv'

with open(path, 'w') as f:
  df_.to_csv(f)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### (5) 저장한 데이터 불러오기

In [ ]:
df2 = pd.read_csv("/content/drive/My Drive/dataset/vgames2_modified.csv").drop(axis=1, columns="Unnamed: 0")
df2

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
1,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
3,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04
...,...,...,...,...,...,...,...,...,...
16578,Ice Age 2: The Meltdown,GC,2006.0,Platform,Vivendi Games,0.15,0.04,0,0.01
16579,Rainbow Islands: Revolution,PSP,2005.0,Action,Rising Star Games,0.01,0,0,0
16580,NBA 2K16,PS3,2015.0,Sports,Take-Two Interactive,0.44,0.19,0.03,0.13
16581,Toukiden: The Age of Demons,PSV,2013.0,Action,Tecmo Koei,0.05,0.05,0.25,0.03
